In [3]:
import pandas as pd

# Assuming ids_df and other_df are your DataFrames
ids_path = '/Users/ryderwishart/text-librarian/data/marble/ids.txt'
ids_df = pd.read_csv(ids_path, sep='\t', header=0, dtype=str)

# other_df is a concat of two TSVs
macula_paths = [
    '/Users/ryderwishart/librarian-api/data/macula/macula-hebrew.tsv',
    '/Users/ryderwishart/librarian-api/data/macula/macula-greek-SBLGNT.tsv'
]

macula_dfs = [pd.read_csv(path, sep='\t', header=0, dtype=str) for path in macula_paths]

other_df = pd.concat(macula_dfs)

print(ids_df.head())
print(other_df.head()) 




         MARBLE_ID       TEXT     VREF
0  H00100100100002        בְּ  GEN 1:1
1  H00100100100004  רֵאשִׁ֖ית  GEN 1:1
2  H00100100100006    בָּרָ֣א  GEN 1:1
3  H00100100100008  אֱלֹהִ֑ים  GEN 1:1
4  H00100100100010       אֵ֥ת  GEN 1:1
          xml:id        ref class       text transliteration after  \
0  o010010010011  GEN 1:1!1  prep        בְּ              bə   NaN   
1  o010010010012  GEN 1:1!1  noun  רֵאשִׁ֖ית         rēʾšiyṯ         
2  o010010010021  GEN 1:1!2  verb    בָּרָ֣א           bārāʾ         
3  o010010010031  GEN 1:1!3  noun  אֱלֹהִ֑ים        ʾĕlōhiym         
4  o010010010041  GEN 1:1!4    om       אֵ֥ת             ʾēṯ         

  strongnumberx stronglemma sensenumber     greek  ... normalized strong case  \
0         0871a         בְּ         NaN        ἐν  ...        NaN    NaN  NaN   
1          7225    רֵאשִׁית           1      ἀρξῇ  ...        NaN    NaN  NaN   
2          1254      בָּרָא           1  ἐποίησεν  ...        NaN    NaN  NaN   
3          0430    אֱ

In [14]:
# Creating the 'vref' column in other_df by stripping '!#' from 'ref'
other_df['vref'] = other_df['ref'].str.split('!').str[0]

# Group ids_df by 'VREF' and 'TEXT' and aggregate 'MARBLE_ID' into a list
grouped_ids_df = ids_df.groupby(['VREF', 'TEXT'])['MARBLE_ID'].apply(list).reset_index()

# Merging the two dataframes based on 'vref' and 'TEXT'
merged_df = pd.merge(other_df, grouped_ids_df,  how='left', 
                     left_on=['vref', 'text'], right_on=['VREF', 'TEXT'])

# Rename the 'MARBLE_ID' column to 'marble_ids'
merged_df.rename(columns={'MARBLE_ID': 'marble_ids'}, inplace=True)

# The 'marble_ids' column will contain lists of matching 'MARBLE_ID's from ids_df

In [15]:
merged_df.head()

,xml:id,ref,class,text,transliteration,after,strongnumberx,stronglemma,sensenumber,greek,...,voice,mood,degree,domain,ln,referent,vref,VREF,TEXT,marble_ids
0,o010010010011,GEN 1:1!1,prep,בְּ,bə,NaN,0871a,בְּ,NaN,ἐν,...,NaN,NaN,NaN,NaN,NaN,NaN,GEN 1:1,GEN 1:1,בְּ,[H00100100100002]
1,o010010010012,GEN 1:1!1,noun,רֵאשִׁ֖ית,rēʾšiyṯ,,7225,רֵאשִׁית,1,ἀρξῇ,...,NaN,NaN,NaN,NaN,NaN,NaN,GEN 1:1,GEN 1:1,רֵאשִׁ֖ית,[H00100100100004]
2,o010010010021,GEN 1:1!2,verb,בָּרָ֣א,bārāʾ,,1254,בָּרָא,1,ἐποίησεν,...,NaN,NaN,NaN,NaN,NaN,NaN,GEN 1:1,GEN 1:1,בָּרָ֣א,[H00100100100006]
3,o010010010031,GEN 1:1!3,noun,אֱלֹהִ֑ים,ʾĕlōhiym,,0430,אֱלֹהִים,1,θεὸς,...,NaN,NaN,NaN,NaN,NaN,NaN,GEN 1:1,GEN 1:1,אֱלֹהִ֑ים,[H00100100100008]
4,o010010010041,GEN 1:1!4,om,אֵ֥ת,ʾēṯ,,0853,אֵת,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,GEN 1:1,GEN 1:1,אֵ֥ת,"[H00100100100010, H00100100100018]"


In [11]:
# print some rows in merged_df where `xml:id` is na
len(merged_df[merged_df['TEXT'].isna()])

183212

In [16]:
# print lengths of all three DataFrames
print(len(ids_df))
print(len(other_df))
print(len(merged_df))

# print some rows in merged_df where `xml:id` is not unique
merged_df[merged_df['xml:id'].duplicated()].head()

580548
605650
605650


,xml:id,ref,class,text,transliteration,after,strongnumberx,stronglemma,sensenumber,greek,...,voice,mood,degree,domain,ln,referent,vref,VREF,TEXT,marble_ids


In [ ]:
# export as a new TSV
merged_df.to_csv('/Users/ryderwishart/text-librarian/data/marble/macula-with-marble-ids.tsv', sep='\t', index=False)